# 十種猴子影像辨識

## 導入模組

In [27]:
using Flux
using Flux.Data: DataLoader
using Flux: @epochs, Conv, MaxPool, onecold, onehotbatch, throttle, logitcrossentropy
using Augmentor
using Statistics
using Images
using JLD

## 載入資料

In [28]:
pl = Resize(128, 128) |>
     Either(1=>FlipX(), 1=>NoOp()) |>
     Either(1=>Zoom(0.9:0.1:1.2), 1=>NoOp()) |>
     CropSize(128, 128)

4-step Augmentor.ImmutablePipeline:
 1.) Resize to 128×128
 2.) Either: (50%) Flip the X axis. (50%) No operation.
 3.) Either: (50%) Zoom by I ∈ {0.9×0.9, 1.0×1.0, 1.1×1.1, 1.2×1.2}. (50%) No operation.
 4.) Crop a 128×128 window around the center

### 載入訓練資料

In [29]:
monkeys = Array{Float64}(undef, 1097, 128, 128, 3)
mon_key = 1
pbase = "training/training/n"
for i in (0:9)
    stri = string(i)
    p = pbase * stri * "/" * "("
    amount = [105, 111, 110, 122, 105, 113, 106, 114, 106, 105]
    for j in (1:amount[i + 1])
        imarray = []
        strj = string(j)
        path = p * strj * ").jpg"
        img = load(path)
        imgg = augment(img, pl)
        mat = channelview(imgg)
        c = copy(mat)
        c2 = zeros(128, 128, 3)
        convert(Array{N0f8, 3}, c2)
        for i in (1:3)
            for j in (1:128)
                for k in (1:128)
                    c2[j, k, i] = copy(convert(N0f8, c[i, j, k]))
                end
            end
        end
        monkeys[mon_key, :, :, :] = c2
        mon_key += 1
    end
end

ArgumentError: ArgumentError: No file exists at given path: training/training/n9/(105).jpg

### 儲存訓練資料

In [30]:
save("monkey_data.jld", "data", monkeys)

### 讀入已儲存的訓練資料

In [31]:
train_x = load("monkey_data.jld")["data"]
train_x = reshape(train_x, 128, 128, 3, :)
train_x = Float32.(train_x)

128×128×3×1097 Array{Float32,4}:
[:, :, 1, 1] =
 0.403922   0.615686    0.984314  …  0.207843    0.027451   0.85098
 0.156863   0.301961    0.780392     0.227451    0.14902    1.0
 0.470588   0.32549     0.32549      0.278431    0.494118   0.231373
 0.996078   0.145098    0.298039     0.427451    0.25098    0.654902
 0.917647   0.74902     0.854902     0.0941176   0.341176   0.443137
 0.423529   0.0823529   0.211765  …  0.729412    0.137255   0.67451
 0.513726   0.00784314  0.756863     0.00392157  0.219608   0.396078
 0.262745   0.0156863   0.866667     0.698039    0.4        0.231373
 0.576471   0.756863    0.223529     0.596078    0.196078   0.729412
 0.227451   0.32549     1.0          0.0156863   0.227451   0.117647
 0.25098    0.0862745   0.321569  …  0.356863    0.368627   0.623529
 0.411765   0.266667    0.686275     0.509804    0.203922   0.0509804
 0.533333   0.282353    0.133333     0.27451     0.286275   0.482353
 ⋮                                ⋱  ⋮                      


In [46]:
#imshow(train_x[:, :, :, 1])

### 載入驗證資料

In [33]:
val_monkeys = Array{Float64}(undef, 272, 128, 128, 3)
val_key = 1
pbase = "validation/validation/n"
for i in (0:9)
    stri = string(i)
    p = pbase * stri * "/" * "("
    amount = [26, 28, 27, 30, 26, 28, 26, 28, 27, 26]
    for j in (1:amount[i + 1])
        imarray = []
        strj = string(j)
        path = p * strj * ").jpg"
        img = load(path)
        imgg = augment(img, pl)
        mat = channelview(imgg)
        c = copy(mat)
        c2 = zeros(128, 128, 3)
        convert(Array{N0f8, 3}, c2)
        for i in (1:3)
            for j in (1:128)
                for k in (1:128)
                    c2[j, k, i] = copy(convert(N0f8, c[i, j, k]))
                end
            end
        end
        val_monkeys[val_key, :, :, :] = c2
        val_key += 1
    end
end

### 儲存驗證資料

In [34]:
save("monkey_validation.jld", "data", val_monkeys)

### 讀入已儲存的驗證資料

In [35]:
test_x = load("monkey_validation.jld")["data"]
test_x = reshape(test_x, 128, 128, 3, :)
test_x = Float32.(test_x)

128×128×3×272 Array{Float32,4}:
[:, :, 1, 1] =
 0.423529   0.443137   0.976471   …  0.396078   0.364706   0.266667
 0.407843   0.243137   0.4           0.211765   0.294118   0.415686
 0.898039   0.278431   0.486275      0.294118   1.0        0.945098
 0.317647   0.360784   0.596078      0.239216   0.933333   0.27451
 0.678431   0.219608   0.662745      0.0196078  0.521569   0.780392
 0.0784314  0.635294   0.8        …  0.054902   0.494118   0.356863
 0.835294   0.705882   0.54902       0.768627   0.831373   0.317647
 0.713726   0.458824   0.643137      0.0431373  0.454902   0.439216
 0.92549    0.313726   0.25098       0.541176   0.596078   0.196078
 0.486275   0.364706   0.415686      0.137255   0.866667   0.235294
 0.772549   0.819608   0.12549    …  0.780392   0.556863   0.45098
 0.8        0.72549    0.611765      0.701961   0.196078   0.270588
 0.294118   0.12549    0.407843      0.423529   0.611765   0.188235
 ⋮                                ⋱  ⋮                     
 0.117647  

### 訓練標籤

In [36]:
train_labels = Array{Int32}(undef, 1097)
key = 1
amount = [105, 111, 110, 122, 105, 113, 106, 114, 106, 105]
for i in (0:9)
    for j in (1:amount[i + 1])
        train_labels[key] = i
        key += 1
    end
end
train_y = onehotbatch(train_labels, 0:9)

10×1097 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  1  1  1  1  1  1  1  1  1  1  1

### 驗證標籤

In [37]:
val_labels = Array{Int32}(undef, 272)
key = 1
amount = [26, 28, 27, 30, 26, 28, 26, 28, 27, 26]
for i in (0:9)
    for j in (1:amount[i + 1])
        val_labels[key] = i
        key += 1
    end
end
test_y = onehotbatch(val_labels, 0:9)

10×272 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  1  1  1  1  1  1  1  1  1  1  1

In [38]:
batchsize = 32
train = DataLoader(train_x, train_y, batchsize = batchsize, shuffle = true)
test = DataLoader(test_x, test_y, batchsize = batchsize)

DataLoader((Float32[0.42352942 0.44313726 … 0.3647059 0.26666668; 0.40784314 0.24313726 … 0.29411766 0.41568628; … ; 0.18039216 0.3372549 … 0.019607844 0.09411765; 0.77254903 0.74509805 … 0.69411767 0.23921569]

Float32[0.72156864 0.46666667 … 0.06666667 0.19215687; 0.29803923 0.8235294 … 0.28235295 0.21960784; … ; 0.27450982 0.29803923 … 0.4392157 0.14901961; 0.7137255 0.3529412 … 0.45490196 0.39607844]

Float32[0.7607843 1.0 … 0.21960784 0.69803923; 0.9529412 0.40392157 … 0.08627451 0.19215687; … ; 0.41568628 0.38431373 … 0.1764706 0.54901963; 0.6 1.0 … 0.23529412 0.20392157]

Float32[0.43529412 0.3882353 … 0.5803922 0.5882353; 0.49411765 0.45490196 … 0.78431374 0.84313726; … ; 0.27450982 0.6431373 … 0.36078432 0.39607844; 0.4745098 0.7607843 … 0.54509807 0.62352943]

Float32[0.5803922 0.33333334 … 0.7372549 0.41960785; 0.29411766 0.92156863 … 0.49019608 0.40784314; … ; 0.36078432 0.50980395 … 0.6156863 0.24705882; 0.30980393 0.03137255 … 0.30588236 0.41960785]

Float32[0.2784314 0.5

## CNN 模型

In [57]:
model_easy = Chain(
    
    BatchNorm(3),
    Conv((3, 3), 3 => 64, pad = (1, 1), relu), # 128x128
    MaxPool((2, 2), stride = (2, 2)), # 64x64

    BatchNorm(64),
    Conv((3, 3), 64 => 128, pad = (1, 1), relu), # 64x64
    MaxPool((2, 2), stride = (2, 2)), # 32x32

    BatchNorm(128),
    Conv((3, 3), 128 => 64, pad = (1, 1), relu), # 32x32
    MaxPool((2, 2), stride = (2, 2)), # 16x16

    BatchNorm(64),
    Conv((3, 3), 64 => 32, pad = (1, 1), relu), # 16x16
    MaxPool((2, 2), stride = (2, 2)), # 8x8
    
    Flux.flatten,
    Dense(2048, 256, relu),
    BatchNorm(256),
    Dense(256, 10),
    softmax
)

Chain(BatchNorm(3), Conv((3, 3), 3=>64, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), BatchNorm(64), Conv((3, 3), 64=>128, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), BatchNorm(128), Conv((3, 3), 128=>64, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), BatchNorm(64), Conv((3, 3), 64=>32, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), flatten, Dense(2048, 256, relu), BatchNorm(256), Dense(256, 10), softmax)

In [58]:
size(train_x)

(128, 128, 3, 1097)

In [59]:
model_easy(train_x[:, :, :, 1:32])

10×32 Array{Float32,2}:
 0.0873409  0.0856567  0.0871958  …  0.0859602  0.0880751  0.0867085
 0.116919   0.114331   0.114934      0.118201   0.11567    0.115781
 0.100349   0.0985155  0.1004        0.0979089  0.0982379  0.0983721
 0.1099     0.111443   0.107595      0.110798   0.109182   0.111184
 0.0961342  0.0947224  0.0975743     0.0930953  0.0974789  0.0949453
 0.0954326  0.0954534  0.0966277  …  0.0937631  0.0960657  0.0950132
 0.11105    0.113377   0.112667      0.114771   0.112569   0.113569
 0.0821126  0.0826165  0.0817951     0.0841097  0.0837102  0.0831852
 0.0955886  0.0972935  0.0957425     0.0950823  0.0947332  0.0971116
 0.105173   0.106591   0.10547       0.10631    0.104277   0.10413

## 損失函數

In [60]:
loss(x, y) = logitcrossentropy(model_easy(x), y)

loss (generic function with 1 method)

In [61]:
accuracy(x, y, model) = mean(onecold(model_easy(x)) .== onecold(y))

accuracy (generic function with 1 method)

## Callback 函式

In [62]:
function callback()
    print("Testing ")
    @show loss(test_x, test_y)
    print("Testing ")
    @show accuracy(test_x, test_y, model_easy)
    println()
    accuracy(test_x, test_y, model_easy) > 0.8 && Flux.stop()
end

callback (generic function with 1 method)

## 模型訓練

In [65]:
epochs = 100
lr_decay = 0.01
lr = 0.01
for i in (1 : epochs)
    println("epoch =" * string(i))
    lr = lr / (1 + lr_decay * (i - 1))
    @epochs 1 Flux.train!(loss, params(model_easy), train, 
                          ADAM(lr, (0.9, 0.8)), cb = throttle(callback, 500))
end

epoch =1


┌ Info: Epoch 1
└ @ Main C:\Users\USER\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


Testing loss(test_x, test_y) = 2.3059244f0
Testing accuracy(test_x, test_y, model_easy) = 0.10294117647058823

epoch =2


┌ Info: Epoch 1
└ @ Main C:\Users\USER\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


Testing loss(test_x, test_y) = 2.221724f0
Testing accuracy(test_x, test_y, model_easy) = 0.23529411764705882

epoch =3


...

┌ Info: Epoch 1
└ @ Main C:\Users\USER\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


Testing loss(test_x, test_y) = 2.1440597f0
Testing accuracy(test_x, test_y, model_easy) = 0.3125

epoch =55


┌ Info: Epoch 1
└ @ Main C:\Users\USER\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


Testing loss(test_x, test_y) = 2.142391f0
Testing accuracy(test_x, test_y, model_easy) = 0.3235294117647059



InterruptException: InterruptException:

## 模型評估

In [66]:
print("Validation Accuracy: $(accuracy(test_x, test_y, model_easy) * 100)%")

Validation Accuracy: 31.61764705882353%